In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.



ageage in years
sex(1 = male; 0 = female)
cpchest pain type
trestbpsresting blood pressure (in mm Hg on admission to the hospital)
cholserum cholestoral in mg/dl
fbs(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
restecgresting electrocardiographic results
thalachmaximum heart rate achieved
exangexercise induced angina (1 = yes; 0 = no)
oldpeakST depression induced by exercise relative to rest
slopethe slope of the peak exercise ST segment
canumber of major vessels (0-3) colored by flourosopy
thal3 = normal; 6 = fixed defect; 7 = reversable defect
target1 or 0

# 0. Importing data and first look
Origin of the dataset


In [8]:
import os
for dirpath, dirnames, filenames in os.walk('.'):
    print(dirpath, filenames)

. ['Heart Diseases - Risk Factors.ipynb', 'LICENSE', 'README.md', '_config.yml']
.\.git ['config', 'description', 'HEAD', 'index', 'packed-refs']
.\.git\hooks ['applypatch-msg.sample', 'commit-msg.sample', 'fsmonitor-watchman.sample', 'post-update.sample', 'pre-applypatch.sample', 'pre-commit.sample', 'pre-push.sample', 'pre-rebase.sample', 'pre-receive.sample', 'prepare-commit-msg.sample', 'update.sample']
.\.git\info ['exclude']
.\.git\logs ['HEAD']
.\.git\logs\refs []
.\.git\logs\refs\heads ['master']
.\.git\logs\refs\remotes []
.\.git\logs\refs\remotes\origin ['HEAD']
.\.git\objects []
.\.git\objects\19 ['39fc623ebcf6bc68b363b58fc955c4d12bd969']
.\.git\objects\25 ['9a24e4d2e66db04470a6b8362a89452328583e']
.\.git\objects\2a ['e9bd011f0a8f760440edd79c64f5d60757e72d']
.\.git\objects\32 ['7b29b4cdc3ff76a958bb6ec77286590a241d37']
.\.git\objects\3f ['a429e297ad1a3e0fc75e5b2a7b3482f926326c']
.\.git\objects\a4 ['d2f963e35a85095c43d043c74d4836f082da02']
.\.git\objects\cb ['192ecd8e9f0e43691

In [9]:
df = pd.read_csv('.\\input\\heart-disease-uci\\heart.csv')
print("Number of samples : %i - Number of features : %i" % (df.shape[0], df.shape[1]))
df.describe()

Number of samples : 303 - Number of features : 14


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [14]:
#!pip install qgrid
! jupyter nbextension enable --py --sys-prefix qgrid
import qgrid
# only required if you have not enabled the ipywidgets nbextension yet
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#to show a df simply use the below:
qgrid.show_grid(df)

Enabling notebook extension qgrid/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## General data quality - missing values
No null values in this -very clean- dataset

In [ ]:
df.info()

In [18]:
data_dict = pd.DataFrame()
#columns=['name', 'type', 'nonNull#', 'nonNull%', 'nunique'], index_col = 'name')
list(df.keys())
SHORTLIST = 5
for col in list(df.keys()):
    uv_list = (list(df[col].unique()))
    uv_list.sort()
    uv_number = df[col].nunique()
    print ("%s - Type : %s - %i Unique values among %s" % (col,df[col].dtype, uv_number, uv_list[:SHORTLIST]))
    data_dict = data_dict.append({'name' : col, 'dtype' : df[col].dtype, 'nonNull#' : uv_number}, ignore_index=True)

age - Type : int64 - 41 Unique values among [29, 34, 35, 37, 38]
sex - Type : int64 - 2 Unique values among [0, 1]
cp - Type : int64 - 4 Unique values among [0, 1, 2, 3]
trestbps - Type : int64 - 49 Unique values among [94, 100, 101, 102, 104]
chol - Type : int64 - 152 Unique values among [126, 131, 141, 149, 157]
fbs - Type : int64 - 2 Unique values among [0, 1]
restecg - Type : int64 - 3 Unique values among [0, 1, 2]
thalach - Type : int64 - 91 Unique values among [71, 88, 90, 95, 96]
exang - Type : int64 - 2 Unique values among [0, 1]
oldpeak - Type : float64 - 40 Unique values among [0.0, 0.1, 0.2, 0.3, 0.4]
slope - Type : int64 - 3 Unique values among [0, 1, 2]
ca - Type : int64 - 5 Unique values among [0, 1, 2, 3, 4]
thal - Type : int64 - 4 Unique values among [0, 1, 2, 3]
target - Type : int64 - 2 Unique values among [0, 1]


In [19]:
data_dict


2
# Add a new row at index k with values provided in list
dfObj.loc['k'] = ['Smriti', 26, 'Bangalore', 'India']


More efficient:

pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)],
          ignore_index=True)

,dtype,name,nonNull#
0,int64,age,41.0
1,int64,sex,2.0
2,int64,cp,4.0
3,int64,trestbps,49.0
4,int64,chol,152.0
5,int64,fbs,2.0
6,int64,restecg,3.0
7,int64,thalach,91.0
8,int64,exang,2.0
9,float64,oldpeak,40.0


In [ ]:
row_max=3
col_max=5
f, axes = plt.subplots(row_max,col_max, figsize=(30, 10)) #, sharex=True)
row=0
col=0
for feat in list(df.keys()):
    sns.distplot(df[feat], ax=axes[row,col], kde=False)
    if col < col_max-1:
        col +=1
    else :
        col = 0
        row += 1

In [ ]:
row_max=3
col_max=5
f, axes = plt.subplots(row_max,col_max, figsize=(30, 10)) #, sharex=True)
row=0
col=0
for feat in list(df.keys()):
    sns.distplot(df.loc[(df['target']==1),feat], ax=axes[row,col], kde=False, label='T1')
    sns.distplot(df.loc[(df['target']==0),feat], ax=axes[row,col], kde=False, label='T0')
    if col < col_max-1:
        col +=1
    else :
        col = 0
        row += 1

# Grouping features by type

In [ ]:
target_feature = ['target'] #1 or 0

categorical_features = [ 'sex',    # (1 = male; 0 = female)
                         'cp',    #chest pain types
                         'fbs',  #(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
                         'restecg', #resting electrocardiographic results
                         'exang', #exercise induced angina (1 = yes; 0 = no)
                         'thal'] # 3 = normal; 6 = fixed defect; 7 = reversable defect 

numerical_features = ['age',  #age in years
                         'trestbps', #resting blood pressure 
                      #(in mm Hg on admission to the hospital)
                         'chol', #serum cholestoral in mg/dl
                         'thalach', #maximum heart rate achieved
                         'oldpeak', #ST depression induced by exercise relative to rest
                         'slope', #the slope of the peak exercise ST segment
                         'ca'] #number of major vessels (0-3) colored by flourosopy


> Analysing repartition of numerical values

In [ ]:
list(df.keys())

In [ ]:
row_max=3
col_max=2
f, axes = plt.subplots(row_max,col_max, figsize=(20, 10), sharex=True)
row=0
col=0
print(axes.shape)
for param in ["trestbps", "thalach", "age","chol", "oldpeak"]:
    sns.scatterplot(x="age", y = param, hue="target", data = df, ax=axes[row,col])
    print(row, col)
    if col < col_max-1:
        col +=1
    else :
        col = 0
        row += 1

In [ ]:

        
row_max=3
col_max=3
f, axes = plt.subplots(row_max,col_max, figsize=(20, 10)) #, sharex=True)
row=0
col=0
for feat in numerical_features:
    sns.distplot(df[feat], ax=axes[row,col])
    if col < col_max-1:
        col +=1
    else :
        col = 0
        row += 1

In [ ]:
plt.figure()
sns.scatterplot(x ="age", y = "thalach",data = df)
sns.regplot(x ="age", y = "thalach",data = df)
plt.show()

In [ ]:
plt.figure();
#sns.scatterplot(x ="thalach", y = "age",hue = "target",data = df)
sns.lmplot(x ="age", y = "thalach",hue = "target",data = df);
#, xlim = (210,70)

In [ ]:
plt.figure(figsize=(10,10))
sns.swarmplot(x="target", y = "thalach", data = df)

In [ ]:
f, axes = plt.subplots(1,4, figsize=(20, 10), sharex=True)
i=0
for param in ["trestbps", "thalach", "age","chol"]:
    sns.swarmplot(x="target", y = param, data = df, ax=axes[i])
    i+=1
    

In [ ]:
row_max=3
col_max=2
f, axes = plt.subplots(row_max,col_max, figsize=(20, 10), sharex=True)
row=0
col=0
print(axes.shape)
for param in ["trestbps", "thalach", "age","chol", "oldpeak"]:
    sns.scatterplot(x="age", y = param, hue="target", data = df, ax=axes[row,col])
    print(row, col)
    if col < col_max-1:
        col +=1
    else :
        col = 0
        row += 1